In [2]:
%load_ext autoreload
%autoreload 2

# import libraries
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pyBigWig
import pybedtools # if on M1 chip mac while installing bedtools make sure conda config --env --set subdir osx-64 (i was on arm-64 and it could not find bedtools!!!) also install rosetta2 /usr/sbin/softwareupdate --install-rosetta --agree-to-license
from pybedtools import BedTool
import numpy as np
import gzip

# change parent directory to use tools

from pathlib import Path
os.chdir(Path.cwd().parent)
from introgression_tools import tools

HOMEDIR='/wynton/home/capra/ychen39/'
BEDTOOLSDIR = '/wynton/home/cbi/shared/software/CBI/bedtools2-2.31.1/bin'
# set bedtools path
pybedtools.helpers.set_bedtools_path(path=f'{BEDTOOLSDIR}')

methods_dirs = ['chen_2020', 'yuan_2021', 'durvasula_2019', 'steinruecken_2018', 'skov_2020', 'schaefer_2021', 'hubisz_2020', 'browning_2018', 'vernot_2016', 'sankararaman_2014', 'sankararaman_2016_1', 'sankararaman_2016_2']


## Generating standardized variant-level files across introgression methods
SNPs are only available for Sankararaman14, Sankararaman16 (1 and 2), S*, Sprime, and Skov20.

We will also make .bed files for each ancestral group.

Here, we use the introgression_tools package to generate standardized variant-level data. Vernot and CRF variant-level introgression files were generated from Bash scripts.

#### Sankararaman 2014

In [5]:
%%bash

# set up directories
INPUTDIR="/wynton/home/capra/ychen39/introgression_methods/data/sankararaman_2014_copy/sankararaman_2014"
OUTPUTDIR="/wynton/home/capra/ychen39/introgression_methods/cleaned/introgression_tools/sankararaman_2014/"

mkdir $OUTPUTDIR

# for each population file
# add column names
echo -e "Chromosome\tEnd\tAlleleScore\tHaplotypeScore\tPopulation" > $OUTPUTDIR/neanderthal_introgressed_variants_pops.bed
# add data from each population file
for pop in $(cat $INPUTDIR/pops.EUR-ASN); do
    echo $pop
    cd $INPUTDIR/"$pop".hapmap/summaries/
    FILES=`ls --color=never *.gz`
    for file in $FILES; do
        echo $file
        # keep entries if their nean probabilities (column 11) are greater than 0.9
        gunzip -c $file | awk -v pop="$pop" '{ if ($10+0 > 0.9) print $2 "\t" $4 "\t" $10 "\t" $11 "\t" pop}' >> $OUTPUTDIR/pops.bed
    done
done
# remove extra file lines except ones starting with numbers (chromosomes), X (for chromosome X), and C (header line)
grep '^[0-9 | X | C]' $OUTPUTDIR/pops.bed > $OUTPUTDIR/neanderthal_introgressed_variants_pops.bed


mkdir: cannot create directory ‘/wynton/home/capra/ychen39/introgression_methods/cleaned/introgression_tools/sankararaman_2014/’: File exists


CEU
chr-10.thresh-90.length-0.00.gz
chr-11.thresh-90.length-0.00.gz
chr-12.thresh-90.length-0.00.gz
chr-13.thresh-90.length-0.00.gz
chr-14.thresh-90.length-0.00.gz
chr-15.thresh-90.length-0.00.gz
chr-16.thresh-90.length-0.00.gz
chr-17.thresh-90.length-0.00.gz
chr-18.thresh-90.length-0.00.gz
chr-19.thresh-90.length-0.00.gz
chr-1.thresh-90.length-0.00.gz
chr-20.thresh-90.length-0.00.gz
chr-21.thresh-90.length-0.00.gz
chr-22.thresh-90.length-0.00.gz
chr-2.thresh-90.length-0.00.gz
chr-3.thresh-90.length-0.00.gz
chr-4.thresh-90.length-0.00.gz
chr-5.thresh-90.length-0.00.gz
chr-6.thresh-90.length-0.00.gz
chr-7.thresh-90.length-0.00.gz
chr-8.thresh-90.length-0.00.gz
chr-9.thresh-90.length-0.00.gz
chr-X.thresh-90.length-0.00.gz
FIN
chr-10.thresh-90.length-0.00.gz
chr-11.thresh-90.length-0.00.gz
chr-12.thresh-90.length-0.00.gz
chr-13.thresh-90.length-0.00.gz
chr-14.thresh-90.length-0.00.gz
chr-15.thresh-90.length-0.00.gz
chr-16.thresh-90.length-0.00.gz
chr-17.thresh-90.length-0.00.gz
chr-18.thr

In [20]:
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants.bg', 
              prefix=False, 
              save_with_header=False,
              colnames=['Chromosome', 'End', 'AlleleScore', 'HaplotypeScore', 'Ancestry'])

In [23]:
# with chr prefix
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/sankararaman_2014_copy/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants_chr_prefix.bg', 
              prefix=True, 
              save_with_header=False,
              colnames=['Chromosome', 'End', 'AlleleScore', 'HaplotypeScore', 'Ancestry'])

In [28]:
# Make EUR variant files
sankararaman_2014 = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants_pops.bed', 
                            sep='\t', low_memory=False)

sankararaman_2014['Ancestry'] = sankararaman_2014['Population'].apply(tools.map_status)

sankararaman_2014_EUR=sankararaman_2014[sankararaman_2014['Ancestry']=='EUR']

tools.save_bed_only(df=sankararaman_2014_EUR, 
         output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants_EUR_chr_prefix.bed', prefix=True)

#### Sankararaman 2016

In [ ]:
# sankararaman 2016 - 1
# Repeat for first directory

# set up directories
INPUTDIR="/wynton/home/capra/ychen39/introgression_methods/data/sankararaman_2016/summaries/1/neandertal"
OUTPUTDIR="/wynton/home/capra/ychen39/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1"

mkdir $OUTPUTDIR

# add column names
echo -e "Chromosome\tEnd\tAlleleScore\tHaplotypeScore\tPopulation" > $OUTPUTDIR/pops.bed
# add data from each population file
for pop in $(cat $INPUTDIR/../../pops); do
    cd $INPUTDIR/"$pop"/summaries/
    FILES=`ls --color=never pred*`
    for file in $FILES; do
        echo $file
        # keep entries if their nean probabilities (column 10) are greater than 0.5
        awk -v pop="$pop" '{ if ($10+0 > 0.5) print $2 "\t" $4 "\t" $10 "\t" $11 "\t" pop}' $file >> $OUTPUTDIR/pops.bed
    done
done

# remove extra file lines except ones starting with numbers (chromosomes), X (for chromosome X), and C (header line)
grep '^[0-9 | X | C]' $OUTPUTDIR/pops.bed > $OUTPUTDIR/neanderthal_introgressed_variants_pops.bed


In [35]:
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants.bg', 
              prefix=False, 
              save_with_header=False)

In [46]:
# with chr prefix
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants_chr_prefix.bg', 
              prefix=True,
              save_with_header=False)

In [37]:
# Make EUR variant files
sankararaman_2016_1 = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants_pops.bed', 
                            sep='\t', low_memory=False)

sankararaman_2016_1['Ancestry'] = sankararaman_2016_1['Population'].apply(tools.map_status)
sankararaman_2016_1_EUR=sankararaman_2016_1[sankararaman_2016_1['Ancestry']=='EUR']

tools.save_bed_only(df=sankararaman_2016_1_EUR, 
         output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants_EUR_chr_prefix.bed', 
         prefix=True)

In [ ]:
# sankararaman 2016 - 2
%%bash

# set up directories
INPUTDIR="/wynton/home/capra/ychen39/introgression_methods/data/sankararaman_2016/summaries/2/neandertal"
OUTPUTDIR="/wynton/home/capra/ychen39/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2"

# add column names
echo -e "Chromosome\tEnd\tAlleleScore\tHaplotypeScore\tPopulation" > $OUTPUTDIR/pops.bed
# add data from each population file
for pop in $(cat $INPUTDIR/../../pops); do
    cd $INPUTDIR/"$pop"/summaries/
    FILES=`ls --color=never pred*`
    for file in $FILES; do
        echo $file
        # keep entries if their nean probabilities from derived alleles (column 10) are greater than 0.5
        awk -v pop="$pop" '{print $1 print $2 "\t" $4 "\t" $10 "\t" $11 "\t" pop}' $file >> $OUTPUTDIR/pops.bed
    done
done

# remove extra file lines except ones starting with numbers (chromosomes), X (for chromosome X), and C (header line)
grep '^[0-9 | X | C]' $OUTPUTDIR/pops.bed > $OUTPUTDIR/neanderthal_introgressed_variants_pops

# sort -k1,1 -k2,2n {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_sorted.bg
# sort -k1,1 -k2,2n {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/original_deserts.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/original_deserts_sorted.bg

In [38]:
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants.bg', 
              prefix=False, 
              save_with_header=False)

In [39]:
# with chr prefix
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_chr_prefix.bg', 
              prefix=True, 
              save_with_header=False)

In [40]:
# Make EUR variant files
sankararaman_2016_2 = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_pops.bed', 
                            sep='\t', low_memory=False)

sankararaman_2016_2['Ancestry'] = sankararaman_2016_2['Population'].apply(tools.map_status)
sankararaman_2016_2_EUR=sankararaman_2016_2[sankararaman_2016_2['Ancestry']=='EUR']

tools.save_bed_only(df=sankararaman_2016_2_EUR, 
         output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_EUR_chr_prefix.bed', prefix=True)

#### Browning 2018

In [2]:
tools.browning_2018_raw_to_bed(input_dir=f'{HOMEDIR}/introgression_methods/data/browning_2018',
                               output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018', 
                               output_file_name='neanderthal_introgressed_variants_pops.bed',
                               archaic='Neanderthal',
                               snp_level=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/browning_2018’: File exists


Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_pops.bed


In [9]:
tools.save_bed_only(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_pops.bed', 
        output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_chr_prefix.bg', prefix=True)

In [10]:
# with chr prefix
tools.save_bed_only(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_chr_prefix.bg', 
              prefix=True,
              save_with_header=False)

In [7]:
# Make EUR variant files
browning_2018 = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_pops.bed', 
                            sep='\t')

browning_2018_EUR=browning_2018[browning_2018['Ancestry']=='EUR']

tools.save_bed_only(df=browning_2018_EUR, 
         output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_EUR_chr_prefix.bed', prefix=True)

#### Skov 2020

In [16]:
tools.skov_2020_raw_to_bed(input_file=f'{HOMEDIR}/introgression_methods/data/skov_2020/41586_2020_2225_MOESM4_ESM.txt',
                                output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020', 
                                output_file_name='neanderthal_introgressed_variants_pops.bed',
                                archaic='Neanderthal',
                                snp_level=True,
                                liftover_path='/wynton/group/capra/bin/liftOver')

Reading liftover chains
Mapping coordinates


In [88]:
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/neanderthal_introgressed_variants_pops.bed', 
        output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/neanderthal_introgressed_variants.bg', prefix=False)

In [12]:
# with chr prefix
tools.save_bed_only(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/neanderthal_introgressed_variants_chr_prefix.bg', 
              prefix=True,
              save_with_header=False)

#### Vernot 2016

In [22]:
%%bash
# set up directories
HOMEDIR=/wynton/home/capra/ychen39/
INPUTDIR=${HOMEDIR}/introgression_methods/data/vernot_2016/introgressed_tag_snp_frequencies
OUTPUTDIR=${HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/

# for SNPs files (not extended LD)
# add data from each population file
for i in ASN EUR PNG SAS; do
    awk -v pop="$i" '{print $1 "\t" $2 "\t" $3 "\t" $4 "\t" $5 "\t" $14 "\t" $15 "\t" $16 "\t" pop}' \
    $INPUTDIR/all_tag_snps."$i".merged.ALL.0.3_R2_cluster.1KG_phase3_essentials.bed >> $OUTPUTDIR/neanderthal_introgressed_variants_pops.bed # correct number of snps, should be 526,225
done
# sort by chromosome
sort -u $OUTPUTDIR/neanderthal_introgressed_variants_pops.bed | sort -k 1,1 -k2,2n > $OUTPUTDIR/tmp; mv $OUTPUTDIR/tmp $OUTPUTDIR/neanderthal_introgressed_variants_pops.bed
# add column names
sed -i '1s/^/Chromosome\tStart\tStop\tAncestralAllele\tDerivedAllele\tNeanderthalBase\tDenisovaBase\tHaplotypeTag\tAncestry\n/' $OUTPUTDIR/neanderthal_introgressed_variants_pops.bed

In [13]:
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_pops.bed', 
        output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants.bg')

In [14]:
# with chr prefix
tools.save_bed_only(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_pops.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_chr_prefix.bg', 
              prefix=True,
              save_with_header=False)

In [18]:
%%bash
# set up directories
HOMEDIR=/wynton/home/capra/ychen39/
INPUTDIR=${HOMEDIR}/introgression_methods/data/vernot_2016/introgressed_tag_snp_frequencies
OUTPUTDIR=${HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/

# for extended LD files
# add data from each population file
for i in ASN EUR PNG SAS; do
    awk -v pop="$i" '{print $1 "\t" $2 "\t" $3 "\t" $4 "\t" pop}' \
    $INPUTDIR/all_tag_snps."$i".merged.ALL.0.3_R2_cluster.1KG_phase3_essentials.bed.extended_LD.sorted >> $OUTPUTDIR/neanderthal_introgressed_variants_pops_extendedLD.bed
done
# sort by chromosome
sort -u $OUTPUTDIR/neanderthal_introgressed_variants_pops_extendedLD.bed | sort -k 1,1 -k2,2n > $OUTPUTDIR/tmp; mv $OUTPUTDIR/tmp $OUTPUTDIR/neanderthal_introgressed_variants_pops_extendedLD.bed
# add column names
sed -i '1s/^/Chromosome\tStart\tStop\tHaplotypeTag\tAncestry\n/' $OUTPUTDIR/neanderthal_introgressed_variants_pops_extendedLD.bed

In [15]:
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_pops_extendedLD.bed', 
        output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_extendedLD.bg')

In [16]:
# with chr prefix
tools.save_bed_only(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_pops_extendedLD.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_extendedLD_chr_prefix.bg', 
              prefix=True,
              save_with_header=False)

In [20]:
%%bash
# set up directories
HOMEDIR=/wynton/home/capra/ychen39/
INPUTDIR=${HOMEDIR}/introgression_methods/data/vernot_2016/introgressed_tag_snp_frequencies
OUTPUTDIR=${HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/

# for median SNPs + extended LD
# add data from each population file
for i in ASN EUR PNG SAS; do
    awk -v pop="$i" '{print $1 "\t" $2 "\t" $3 "\t" $11 "\t" $12 "\t" pop}' \
    $INPUTDIR/all_tag_snps."$i".merged.ALL.0.3_R2_cluster.1KG_phase3_essentials.median_af.bed.extended_LD >> $OUTPUTDIR/neanderthal_introgressed_variants_pops_medianextendedLD.bed
done
# sort by chromosome
sort -u $OUTPUTDIR/neanderthal_introgressed_variants_pops_medianextendedLD.bed | sort -k 1,1 -k2,2n > $OUTPUTDIR/tmp; mv $OUTPUTDIR/tmp $OUTPUTDIR/neanderthal_introgressed_variants_pops_medianextendedLD.bed
# add column names
sed -i '1s/^/Chromosome\tStart\tStop\tSNPsOnHaplotype\tLength\tAncestry\n/' $OUTPUTDIR/neanderthal_introgressed_variants_pops_medianextendedLD.bed

In [17]:
tools.save_bed_only(full_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_pops_medianextendedLD.bed', 
        output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_medianextendedLD.bg')

In [18]:
# with chr prefix
tools.save_bed_only(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_pops_medianextendedLD.bed', 
              output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_medianextendedLD_chr_prefix.bg', 
              prefix=True,
              save_with_header=False)

In [41]:
# Make EUR variant files
vernot_2016 = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_pops.bed', 
                            sep='\t')

vernot_2016_EUR=vernot_2016[vernot_2016['Ancestry']=='EUR']

tools.save_bed_only(df=vernot_2016_EUR, 
         output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_EUR_chr_prefix.bed', prefix=True)

### Generate variant overlap file

In [29]:
# add column of 1s for a "score" to make proper bedgraphs

!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_score.bg
!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_score.bg
!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_medianextendedLD.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_medianextendedLD_score.bg
!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_extendedLD.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_extendedLD_score.bg
!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants_score.bg
!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants_score.bg
!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_score.bg
!sed 's/$/\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/neanderthal_introgressed_variants.bg > \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/neanderthal_introgressed_variants_score.bg

In [ ]:
# bedtools requires '#' to be at the front of each header, but not using a header here
# create union file
!$BEDTOOLSDIR/bedtools unionbedg -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/neanderthal_introgressed_variants_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_medianextendedLD_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_variants_extendedLD_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/neanderthal_introgressed_variants_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/neanderthal_introgressed_variants_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/neanderthal_introgressed_variants_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/neanderthal_introgressed_variants_score.bg \
    -header -names sprime vernot_2016 vernot_2016_extendedLD vernot_2016_medianextendedLD sankararaman_2014 sankararaman_2016_1 sankararaman_2016_2 skov_2020  > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_variants.bed

In [ ]:
overlap = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_variants.bed', sep='\t', low_memory=False)
# replace 0s with NA
overlap.replace(0, np.nan, inplace=True)
# add length data
overlap['length'] = overlap['end'] - overlap['start']
overlap

overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_variants.bed', sep='\t', index=False)

# save bed file version of all identified introgressed segments across methods
overlap[['chrom', 'start', 'end']].to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only_variants.bed', sep='\t', index=False)

# first, convert dataframe to boolean values
# exclude for boolean
exclude_columns = ['chrom', 'start', 'end', 'length']
# convert to boolean (exclude columns specified)
boolean_overlap = overlap.copy()
boolean_overlap[boolean_overlap.columns.difference(exclude_columns)] = boolean_overlap[boolean_overlap.columns.difference(exclude_columns)].applymap(lambda x: pd.notna(x))
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_variants.bed', sep='\t', index=False)
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_no_header_variants.bed', sep='\t', index=False, header=False)

# drop chrom, start, end
boolean_overlap = boolean_overlap.drop(['chrom', 'start', 'end'], axis=1)
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_no_bed_variants', sep='\t', index=False)
boolean_overlap

,sprime,vernot_2016,vernot_2016_extendedLD,vernot_2016_medianextendedLD,sankararaman_2014,sankararaman_2016_1,sankararaman_2016_2,skov_2020,length
0,True,False,False,False,False,False,False,False,1
1,True,False,False,False,False,False,False,False,1
2,True,False,False,False,False,False,False,False,1
3,True,False,False,False,False,False,False,False,1
4,True,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...
1617413,False,False,False,False,False,True,False,False,1
1617414,False,False,False,False,False,True,False,False,1
1617415,False,False,False,False,False,True,False,False,1
1617416,False,False,False,False,False,True,False,False,1


## Generating standardized fragment files across introgression methods

Here, we use the introgression_tools package to generate standardized individual-level and superpopulation-level introgression data.

Additionally, we will generate "2 Percent" files for individual-level data: we will take the top introgressed regions for each person, consisting of 2% of each individual' genome, so we can repeat individual-level analysis subsetted to "confident" regions and controlling for the same amount of introgression across people.

First, we will convert all raw outputs to standardized .bed and .bg files for later use.

### IBDMix

In [8]:
tools.chen_2020_raw_to_bed(input_path=f'{HOMEDIR}/introgression_methods/data/ibdmix_chen_et_al_2020/Neanderthal sequence in 1000 genome.50kb.txt', 
                           output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/', 
                           output_file_name='individual_nean_chen2020_frag.bed', 
                           individual=True, population=False)

tools.chen_2020_raw_to_bed(input_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/individual_nean_chen2020_frag.bed', 
                           output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/', 
                           output_file_name='superpop_nean_chen2020_frag.bed', 
                           population=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020//bedgraphs’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020//bedgraphs’: File exists


Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020//superpop_nean_chen2020_frag.bed


In [23]:
tools.chen_2020_raw_to_bed(input_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/individual_nean_chen2020_frag.bed', 
                           output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/', 
                           output_file_name='superpop_nean_chen2020_frag.bed', 
                           population=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020//bedgraphs’: File exists


EAS
merged
EUR
merged
AMR
merged
SAS
merged
AFR
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020//superpop_nean_chen2020_frag.bed


In [ ]:
# also save separate dataframe with African admixed individuals only
ibdmix = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/ibdmix_chen2020/individual_introgressed_chen2020_frag.bed',
                     sep='\t')
ibdmix_afr_admix = ibdmix[ibdmix.Population.isin(['ACB', 'ASW'])][['Chromosome', 'Start', 'End', 'Score']]
# print the AFR ids
print('IBDMix AFR (including admixed) individuals:', len(ibdmix[ibdmix.Population.isin(['ACB', 'ASW'])].ID.unique()))
ibdmix_afr_admix.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_admix.bed', sep='\t', index=False, header=False)
ibdmix_afr = ibdmix[ibdmix.Population.isin(['LWK', 'GWD', 'MSL', 'YRI', 'ESN'])][['Chromosome', 'Start', 'End', 'Score']]

# print the AFR ids
print('IBDMix AFR (no admixed) individuals:', len(ibdmix[ibdmix.Population.isin(['LWK', 'GWD', 'MSL', 'YRI', 'ESN'])].ID.unique()))

ibdmix_afr.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr.bed', sep='\t', index=False, header=False)


# create merged file
!sort -k1,1 -k2,2n {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_admix.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_admix_sorted.bed
!$BEDTOOLSDIR/bedtools merge -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_admix_sorted.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_admix_merged_sorted.bed
!rm {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_admix.bed {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_admix_sorted.bed

!sort -k1,1 -k2,2n {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_sorted.bed
!$BEDTOOLSDIR/bedtools merge -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_sorted.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_merged_sorted.bed
!rm {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr.bed {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/afr_sorted.bed

In [ ]:
print('IBDMix AFR (no admixed) individuals:', len(ibdmix[ibdmix.Population.isin(['LWK', 'GWD', 'MSL', 'YRI', 'ESN'])].ID.unique()))
print('IBDMix AFR (including admixed) individuals:', len(ibdmix[ibdmix.Population.isin(['ACB', 'ASW'])].ID.unique()) + len(ibdmix[ibdmix.Population.isin(['LWK', 'GWD', 'MSL', 'YRI', 'ESN'])].ID.unique()))

IBDMix AFR (no admixed) individuals: 504
IBDMix AFR (including admixed) individuals: 661


### ArchaicSeeker2

In [3]:
tools.yuan_2021_raw_to_bed(input_dir_list=[f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/KGP/EastAsia', 
                                       f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/KGP/SouthAsia', 
                                       f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/KGP/WestEurasia',
                                       f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/SGDP'],
                           output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021', 
                           output_file_name='individual_nean_yuan2021_frag.bed', 
                           archaic='Neanderthal',
                           individual=True)

tools.yuan_2021_raw_to_bed(file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/individual_nean_yuan2021_frag.bed',
                           output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/', 
                           output_file_name='superpop_nean_yuan2021_frag.bed', 
                           archaic='Neanderthal',
                           population=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/yuan_2021’: File exists
/wynton/home/capra/ychen39/.conda/envs/jupyter/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/yuan_2021//bedgraphs’: File exists


EAS
merged
SAS
merged
EUR
merged
Oceania
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/yuan_2021//superpop_nean_yuan2021_frag.bed


In [2]:
## retry, removed "contains" to exact match for "Neanderthal" and "Denisovan"

tools.yuan_2021_raw_to_bed(input_dir_list=[f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/KGP/EastAsia', 
                                       f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/KGP/SouthAsia', 
                                       f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/KGP/WestEurasia',
                                       f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/IntrogressedSeg/SGDP'],
                           output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/debugging/', 
                           output_file_name='individual_nean_yuan2021_frag.bed', 
                           archaic='Neanderthal',
                           individual=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/yuan_2021/debugging/’: File exists
/wynton/home/capra/ychen39/.conda/envs/jupyter/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


### ArchIE

In [4]:
# using an increased threshold so we get ~2% introgression per person
tools.durvasula_2019_raw_to_bed(input_dir=f'{HOMEDIR}/introgression_methods/data/archie_durvasula2019/CEU-predictions',
                                ancestry='EUR',
                                output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019', 
                                output_file_name='individual_nean_durvasula2019_frag_higher_threshold.bed', 
                                individual=True,
                                threshold=0.9984,
                                keep_old_score=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019’: File exists


Reading CEU.chr22-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr6-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr15-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr8-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr10-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr12-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr21-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr19-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr9-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr17-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr18-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr7-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr2-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr3-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr20-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr4-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr16-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr1-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr14-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr5-prediction.bed.gz-trim

In [2]:
# using the threshold reported in the paper
tools.durvasula_2019_raw_to_bed(input_dir=f'{HOMEDIR}/introgression_methods/data/archie_durvasula2019/CEU-predictions',
                                ancestry='EUR',
                                output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019', 
                                output_file_name='individual_nean_durvasula2019_frag_thresholded_0.862.bed', 
                                individual=True,
                                threshold=0.862,
                                keep_old_score=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019’: File exists


Reading CEU.chr22-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr6-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr15-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr8-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr10-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr12-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr21-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr19-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr9-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr17-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr18-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr7-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr2-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr3-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr20-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr4-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr16-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr1-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr14-prediction.bed.gz-trimmed.bed.gz
Reading CEU.chr5-prediction.bed.gz-trim

In [5]:
tools.durvasula_2019_raw_to_bed(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019',
                                ancestry='EUR',
                                file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/individual_nean_durvasula2019_frag_higher_threshold.bed',
                                output_file_name='superpop_nean_durvasula2019_frag.bed', 
                                population=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs’: File exists


EUR
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019/superpop_nean_durvasula2019_frag.bed


cp: target 'union_merged.bg' is not a directory


In [3]:
tools.durvasula_2019_raw_to_bed(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019',
                                ancestry='EUR',
                                file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/individual_nean_durvasula2019_frag.bed',
                                output_file_name='superpop_nean_durvasula2019_frag_thresholded_0.862.bed', 
                                population=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs’: File exists


EUR
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/durvasula_2019/superpop_nean_durvasula2019_frag_thresholded_0.862.bed


### DICAL-ADMIX

In [50]:
#tools.steinruecken_2018_raw_to_bed(input_dir=f'{HOMEDIR}/introgression_methods/data/steinrucken_2018/called_tracts',
                                output_dir=f'{HOMEDIR}/introgression_methods/introgression_tools/test/dicaladmix/', 
                                output_file_name='individual_nean_introgressed_steinrucken2018_frag.bed',
                                prob_dir= f'{HOMEDIR}/introgression_methods/data/dical-steinrucken_2018/posterior_probabilities',
                                input_tar_gz='CEU_lax_chr1_beds.tar.gz',
                                individual=True) 

## did not run this, used bash script instead. 
## already generated individual=level file with posterior probabilities

/Users/yaenchen/Library/CloudStorage/OneDrive-NortheasternUniversity/UCSF/CapraLab//introgression_methods/data/dical-admix_steinrucken_et_al_2018/called_tracts/CEU_lax_chr1_beds.tar.gz
/Users/yaenchen/Library/CloudStorage/OneDrive-NortheasternUniversity/UCSF/CapraLab//introgression_methods/data/dical-admix_steinrucken_et_al_2018/called_tracts/CEU_lax_chr1_beds.tar.gz


mkdir: /Users/yaenchen/Library/CloudStorage/OneDrive-NortheasternUniversity/UCSF/CapraLab//introgression_methods/introgression_tools/test/dicaladmix/: File exists


In [39]:
tools.steinruecken_2018_raw_to_bed(file_path=f'{HOMEDIR}/introgression_methods/cleaned/steinruecken_2018/individual_introgressed_steinruecken2018_frag.bed',
                                output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018', 
                                output_file_name='superpop_nean_steinruecken2018_frag.bed',
                                population=True) 

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/steinruecken_2018/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs’: File exists


CEU
merged
CHBS
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/steinruecken_2018/superpop_nean_steinruecken2018_frag.bed


### Skov2020

In [27]:
tools.skov_2020_raw_to_bed(input_file=f'{HOMEDIR}/introgression_methods/data/skov_2020/hg38_to_hg19_liftover/skov_2020_lifted.bed',
                                output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020', 
                                output_file_name='superpop_nean_skov2020_frag.bed',
                                population=True,
                                archaic='Neanderthal')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/skov_2020/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs’: File exists


Icelandic
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/skov_2020/superpop_nean_skov2020_frag.bed


In [84]:
# also clean variants file
#awk '{print $1,$2,$2+1}' {HOMEDIR}/introgression_methods/data/skov_2020/41586_2020_2225_MOESM4_ESM.txt > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/snps.bed

skov_2020_variants = pd.read_csv(f'{HOMEDIR}/introgression_methods/data/skov_2020/41586_2020_2225_MOESM4_ESM.txt', sep='\t', low_memory=False)
# add end position
skov_2020_variants['start'] = skov_2020_variants['pos'] - 1
#print(skov_2020_variants.snptype.unique())
# keep only DAV snps
skov_2020_variants = skov_2020_variants[(skov_2020_variants['snptype'] == 'DAVsnp') | (skov_2020_variants['snptype'] == 'linkedDAVsnp')]
# rename pos to end
skov_2020_variants.rename(columns={'pos': 'end'}, inplace=True)
# keep only bed positions
skov_2020_variants_bed = skov_2020_variants[['chrom', 'start', 'end']]
skov_2020_variants_bed
# save to file -- version with chr prefix for liftover
skov_2020_variants_bed.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/dav_and_linked_snps_hg38_chr_prefix.bg', header=False, sep='\t', index=False)
#drop 'chr' prefix
skov_2020_variants_bed['chrom'] = skov_2020_variants_bed['chrom'].str.replace('chr', '')
# save to file
skov_2020_variants_bed.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/dav_and_linked_snps_hg38.bg', header=False, sep='\t', index=False)

### SARGE

In [14]:
sarge = pd.read_csv(f'{HOMEDIR}/introgression_methods/data/sarge_schaefer2021/sarge_admixed_haps_hs37d5.bed', sep='\t')
len(sarge['hap'][sarge['pop'] == 'Africa'].unique()) / 2

32.0

In [17]:
tools.schaefer_2021_raw_to_bed(input_file=f'{HOMEDIR}/introgression_methods/data/sarge_schaefer2021/sarge_admixed_haps_hs37d5.bed',
                                output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/', 
                                output_file_name='individual_nean_schaefer2021_frag.bed',
                                individual=True,
                                archaic='Neanderthal')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/schaefer_2021/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/schaefer_2021//bedgraphs’: File exists


In [28]:
tools.schaefer_2021_raw_to_bed(input_file=f'{HOMEDIR}/introgression_methods/data/sarge_schaefer2021/sarge_admixed_haps_hs37d5.bed',
                                output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/', 
                                output_file_name='superpop_nean_schaefer2021_frag.bed',
                                population=True,
                                archaic='Neanderthal')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/schaefer_2021/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/schaefer_2021//bedgraphs’: File exists


Oceania
merged
WestEurasia
merged
EastAsia
merged
Africa
merged
Africa2
merged
America
merged
CentralAsiaSiberia
merged
SouthAsia
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/schaefer_2021//superpop_nean_schaefer2021_frag.bed


### ARGWeaver-D

In [26]:
## Process individual files first
tools.hubisz_2020_raw_to_bed(input_path=f'{HOMEDIR}/introgression_methods/data/argweaver-d_hubisz_et_al_2020/ooaM1A/Mandenka_2F.bed', 
                               ancestry='Africa', 
                               ID='LP6005441-DNA_F07',
                               archaic='Neanderthal',
                               output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020',
                               output_file_name='mandenka_individual_nean_hubisz2020_frag.bed',
                               individual=True)

tools.hubisz_2020_raw_to_bed(input_path=f'{HOMEDIR}/introgression_methods/data/argweaver-d_hubisz_et_al_2020/ooaM1A/Papuan_1F.bed', 
                               ancestry='Oceania', 
                               ID='LP6005441-DNA_B10',
                               archaic='Neanderthal',
                               output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020',
                               output_file_name='papuan_individual_nean_hubisz2020_frag.bed',
                               individual=True)

tools.hubisz_2020_raw_to_bed(input_path=f'{HOMEDIR}/introgression_methods/data/argweaver-d_hubisz_et_al_2020/ooaM1A/Khomani_San_1F.bed', 
                               ancestry='Africa', 
                               ID='LP6005677-DNA_D03',
                               archaic='Neanderthal',
                               output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020',
                               output_file_name='khomani_san_individual_nean_hubisz2020_frag.bed',
                               individual=True)

tools.hubisz_2020_raw_to_bed(input_path=f'{HOMEDIR}/introgression_methods/data/argweaver-d_hubisz_et_al_2020/ooaM1A/Basque_2F.bed', 
                               ancestry='WestEurasia', 
                               ID='LP6005441-DNA_D02',
                               archaic='Neanderthal',
                               output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020',
                               output_file_name='basque_individual_nean_hubisz2020_frag.bed',
                               individual=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020’: File exists


Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/mandenka_individual_nean_hubisz2020_frag.bed
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/papuan_individual_nean_hubisz2020_frag.bed
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/khomani_san_individual_nean_hubisz2020_frag.bed
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/basque_individual_nean_hubisz2020_frag.bed


mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs’: File exists


Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/superpop_nean_hubisz2020_frag.bed


In [29]:
## Process population files, concatenate
tools.hubisz_2020_raw_to_bed(individual_files=[f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/mandenka_individual_nean_hubisz2020_frag.bed',
                                               f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/papuan_individual_nean_hubisz2020_frag.bed',
                                               f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/khomani_san_individual_nean_hubisz2020_frag.bed',
                                               f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/basque_individual_nean_hubisz2020_frag.bed'], 
                               output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020',
                               output_file_name='superpop_nean_hubisz2020_frag.bed',
                               population=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs’: File exists


Africa
merged
Oceania
merged
WestEurasia
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/hubisz_2020/superpop_nean_hubisz2020_frag.bed


### Sprime

In [38]:
tools.browning_2018_raw_to_bed(input_dir=f'{HOMEDIR}/introgression_methods/data/browning_2018',
                               output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018', 
                               output_file_name='superpop_nean_browning2018_frag.bed',
                               archaic='Neanderthal')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/browning_2018’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs’: File exists


chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
EAS
merged
SAS
merged
EUR
merged
AMR
merged
Oceania
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/browning_2018/superpop_nean_browning2018_frag.bed


### S*

In [ ]:
%%bash

# set up directories
HOMEDIR=/wynton/home/capra/ychen39/
INPUTDIR=${HOMEDIR}/introgression_methods/data/vernot_2016/introgressed_haplotypes
OUTPUTDIR=${HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/vernot_2016_round2

# for neanderthal
# add data from each population file
for i in EAS EUR PNG SAS; do
    awk -v pop="$i" '{print $1 "\t" $2 "\t" $3 "\t" $5 "\t" pop}' \
    $INPUTDIR/LL.callset"$i".mr_0.99.neand_calls_by_hap.bed.merged.by_chr.bed >> $OUTPUTDIR/neanderthal_introgressed_haplotypes_individual.bed
done

# for denisovan
# add data from each population file
for i in PNG; do
    awk -v pop="$i" '{print $1 "\t" $2 "\t" $3 "\t" $5 "\t" pop}' \
    $INPUTDIR/LL.callset"$i".mr_0.99.den_calls_by_hap.bed.merged.by_chr.bed >> $OUTPUTDIR/denisovan_introgressed_haplotypes_individual.bed # correct number of snps, should be 526,225
done

# sort by chromosome
sort -u $OUTPUTDIR/neanderthal_introgressed_haplotypes_individual.bed | sort -k 1,1 -k2,2n > $OUTPUTDIR/tmp; mv $OUTPUTDIR/tmp $OUTPUTDIR/neanderthal_introgressed_haplotypes_individual.bed
sort -u $OUTPUTDIR/denisovan_introgressed_haplotypes_individual.bed | sort -k 1,1 -k2,2n > $OUTPUTDIR/tmp; mv $OUTPUTDIR/tmp $OUTPUTDIR/denisovan_introgressed_haplotypes_individual.bed


In [13]:
tools.vernot_2016_raw_to_bed(raw_introgressed_haplotypes_dir=f'{HOMEDIR}/introgression_methods/data/vernot_2016/introgressed_haplotypes',
    introgressed_haplotypes_file=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_haplotypes_individual.bed', 
                             output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/', 
                             output_file_name='neanderthal_introgressed_haplotypes_individual.bed',
                             archaic='Neanderthal', individual_level=True)

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/vernot_2016//tmp/’: File exists


In [8]:
tools.vernot_2016_raw_to_bed(raw_introgressed_haplotypes_dir=f'{HOMEDIR}/introgression_methods/data/vernot_2016/introgressed_haplotypes',
                             introgressed_haplotypes_file=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/neanderthal_introgressed_haplotypes_individual.bed', 
                             output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/', 
                             output_file_name='superpop_nean_vernot2016_frag.bed',
                             archaic='Neanderthal')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/vernot_2016/’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/vernot_2016//bedgraphs’: File exists


EAS
merged
SAS
merged
EUR
merged
PNG
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/vernot_2016//superpop_nean_vernot2016_frag.bed


In [ ]:
tools.vernot_2016_raw_to_bed_snp(snp_input_file=f'{HOMEDIR}/introgression_methods/cleaned/vernot_2016/ancestral_introgressed_variants.bed', 
                             extended_ld_input_file=f'{HOMEDIR}/introgression_methods/cleaned/vernot_2016/ancestral_introgressed_variants_extendedLD.bed', 
                             hap_input_file=f'{HOMEDIR}/introgression_methods/cleaned/vernot_2016/ancestral_introgressed_variants_medianextendedLD.bed', 
                             output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/', 
                             output_file_name='superpop_nean_vernot2016_frag_round2.bed',
                             archaic='Neanderthal') # fixed exact match for derived/archaic alleles, changed to isin

In [11]:
tools.vernot_2016_raw_to_bed_snp(snp_input_file=f'{HOMEDIR}/introgression_methods/cleaned/vernot_2016/ancestral_introgressed_variants.bed', 
                             extended_ld_input_file=f'{HOMEDIR}/introgression_methods/cleaned/vernot_2016/ancestral_introgressed_variants_extendedLD.bed', 
                             hap_input_file=f'{HOMEDIR}/introgression_methods/cleaned/vernot_2016/ancestral_introgressed_variants_medianextendedLD.bed', 
                             output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/denisovan', 
                             output_file_name='superpop_den_vernot2016_frag_round2.bed',
                             archaic='Denisovan') # fixed exact match for derived/archaic alleles, changed to isin

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/vernot_2016/vernot_2016_round2/denisovan’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/vernot_2016/vernot_2016_round2/denisovan/bedgraphs’: File exists


SAS
merged
ASN
merged
PNG
merged
EUR
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/vernot_2016/vernot_2016_round2/denisovan/superpop_den_vernot2016_frag_round2.bed


### CRF 2014

In [35]:
tools.sankararaman_2014_raw_to_bed(snp_input_file=f'{HOMEDIR}/introgression_methods/cleaned/sankararaman_2014_copy/neanderthal_introgressed_variants_pops.bed', 
                             hap_input_file=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/1kgids_individual_neanderthal_introgressed_fragments_pops.bed', 
                             output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014', 
                             output_file_name='superpop_nean_sankararaman2014_frag.bed')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2014’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs’: File exists


EUR
merged
EAS
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2014/superpop_nean_sankararaman2014_frag.bed


### CRF 2016

In [33]:
tools.sankararaman_2016_raw_to_bed(hap_input_file=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/sgdp_individual_neanderthal_introgressed_fragments_pops.bed', 
                             output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1', 
                             output_file_name='superpop_nean_sankararaman2016_1_frag.bed')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2016_1’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs’: File exists


AMR
merged
CentralAsia
merged
EAS
merged
Oceania
merged
SAS
merged
EUR
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/superpop_nean_sankararaman2016_1_frag.bed


In [34]:
tools.sankararaman_2016_raw_to_bed(hap_input_file=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/sgdp_individual_neanderthal_introgressed_fragments_pops.bed', 
                             output_dir=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2', 
                             output_file_name='superpop_nean_sankararaman2016_2_frag.bed')

mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2016_2’: File exists
mkdir: cannot create directory ‘/wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs’: File exists


AMR
merged
CentralAsia
merged
EAS
merged
Oceania
merged
SAS
merged
EUR
merged
Output .bed file is located at /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/superpop_nean_sankararaman2016_2_frag.bed


## Create union file. This file is generated from the bedgraphs and need to have a # in the first header line, which we did in the previous section


In [6]:
# bedtools requires '#' to be at the front of each header

!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/vernot_2016_round2/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/union_merged_header.bg
!sed '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/union_merged_highest_score.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/union_merged_header.bg

# Create union file.
!$BEDTOOLSDIR/bedtools unionbedg -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/union_merged_header.bg \
    -header -names ibdmix argweaverd archaicseeker archie sprime vernot_2016 sankararaman_2014 sankararaman_2016_1 sankararaman_2016_2 skov_2020 steinruecken_2018 sarge > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods.bed

In [ ]:
# make overlap file with only bed and chr prefix: overlap_bed_only_no_header_chr_prefix.bg

# only keep chrom, start, stop
!cut -f1,2,3 {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_bed_only.bed
# remove header
!sed -e '1d' -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_bed_only.bed
# add chr prefix
!sed -e 's/^/chr/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_bed_only.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only_no_header_chr_prefix.bg

# create merged file. This file is generated from the bedgraphs and need to have a # in the first header line, which we did in the previous section
!$BEDTOOLSDIR/bedtools merge -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only_no_header_chr_prefix.bg \
    > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/merged_all_methods.bed

In [ ]:
overlap = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods.bed', sep='\t', low_memory=False)
# replace 0s with NA
overlap.replace(0, np.nan, inplace=True)
# add length data
overlap['length'] = overlap['end'] - overlap['start']
overlap

overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods.bed', sep='\t', index=False)

# save bed file version of all identified introgressed segments across methods
overlap[['chrom', 'start', 'end']].to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only.bed', sep='\t', index=False)

# first, convert dataframe to boolean values
# exclude for boolean
exclude_columns = ['chrom', 'start', 'end', 'length']
# convert to boolean
boolean_overlap = overlap.copy()
boolean_overlap[boolean_overlap.columns.difference(exclude_columns)] = boolean_overlap[boolean_overlap.columns.difference(exclude_columns)].applymap(lambda x: pd.notna(x))
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap.bed', sep='\t', index=False)
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_no_header.bed', sep='\t', index=False, header=False)

# drop chrom, start, end
boolean_overlap = boolean_overlap.drop(['chrom', 'start', 'end'], axis=1)
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_no_bed', sep='\t', index=False)
boolean_overlap

,ibdmix,argweaverd,archaicseeker,archie,sprime,vernot_2016,sankararaman_2014,sankararaman_2016_1,sankararaman_2016_2,skov_2020,steinruecken_2018,sarge,length
0,False,False,True,False,False,False,False,False,False,False,False,False,4580
1,False,False,True,False,False,False,False,False,False,False,False,False,5610
2,False,False,True,False,False,False,False,False,False,False,False,False,4454
3,False,False,True,False,False,False,False,False,False,False,False,False,22746
4,False,False,True,False,False,False,False,False,False,False,False,False,9495
...,...,...,...,...,...,...,...,...,...,...,...,...,...
370118,False,False,False,False,False,False,False,True,False,False,False,False,109687
370119,False,False,False,False,False,False,False,True,False,False,False,False,75368
370120,False,False,False,False,False,False,False,True,False,False,False,False,37507
370121,False,False,False,False,False,False,False,True,False,False,False,False,3066


In [9]:
boolean_overlap[boolean_overlap['archie']==True].length.sum()

793900000

#### no argweaver

In [ ]:
## REGION LEVEL FILE
# Create union file without ARGWeaver-D (hubisz 2020)
!$BEDTOOLSDIR/bedtools unionbedg -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/union_merged_header.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/union_merged_header.bg \
    -header -names ibdmix archaicseeker archie sprime vernot_2016 sankararaman_2014 sankararaman_2016_1 sankararaman_2016_2 skov_2020 steinruecken_2018 sarge > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_no_argweaver.bed

# only keep chrom, start, stop
!cut -f1,2,3 {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_no_argweaver.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_no_argweaver_bed_only.bed
# remove header
!sed -e '1d' -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_no_argweaver_bed_only.bed
# add chr prefix
!sed -e 's/^/chr/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_no_argweaver_bed_only.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only_no_argweaver_no_header_chr_prefix.bg

# Create merged file. This file is generated from the bedgraphs and need to have a # in the first header line, which we did in the previous section
!$BEDTOOLSDIR/bedtools merge -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only_no_argweaver_no_header_chr_prefix.bg \
    > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/merged_all_methods_no_argweaver.bed

## Region level
!{BEDTOOLSDIR}/bedtools intersect -a {HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only_no_argweaver_no_header_chr_prefix.bg \
    -b {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/sorted_union_merged_chr_prefix.bg \
    -C -names ibdmix archaicseeker archie sprime vernot_2016 sankararaman_2014 sankararaman_2016_1 sankararaman_2016_2 skov_2020 steinruecken_2018 sarge \
   > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/region_intersect_for_boolean_no_argweaver.bed

Error: Unable to open file /wynton/home/capra/ychen39//introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/sorted_union_merged_chr_prefix.bg. Exiting.


In [8]:
## BASE PAIRS LEVEL
overlap = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_no_argweaver.bed', sep='\t', low_memory=False)
# replace 0s with NA
overlap.replace(0, np.nan, inplace=True)
# add length data
overlap['length'] = overlap['end'] - overlap['start']
overlap

overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_no_argweaver.bed', sep='\t', index=False)

# save bed file version of all identified introgressed segments across methods
overlap[['chrom', 'start', 'end']].to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_no_argweaver_bed_only.bed', sep='\t', index=False)

# first, convert dataframe to boolean values
# List of columns to exclude
exclude_columns = ['chrom', 'start', 'end', 'length']
# Convert non-NA values to True and NA values to False, excluding specified columns
boolean_overlap = overlap.copy()
boolean_overlap[boolean_overlap.columns.difference(exclude_columns)] = boolean_overlap[boolean_overlap.columns.difference(exclude_columns)].applymap(lambda x: pd.notna(x))
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_no_argweaver.bed', sep='\t', index=False)
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_no_argweaver_no_header.bed', sep='\t', index=False, header=False)

# drop chrom, start, end
boolean_overlap = boolean_overlap.drop(['chrom', 'start', 'end'], axis=1)
boolean_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_no_argweaver_no_bed', sep='\t', index=False)
boolean_overlap

## REGION LEVEL

# # convert dataframe to boolean values
# # read in regions overlap file that takes into account different loci by different algorithms
# region_overlap = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/region_intersect_for_boolean_no_argweaver.bed', sep='\t', low_memory=False, names=['chr', 'start', 'end', 'method', 'regions'])
# # Group by chrom, start, end and 
# grouped = region_overlap.groupby(['chr', 'start', 'end', 'method']).sum().reset_index()
# # Pivot table to turn methods -> columns
# pivot_table = grouped.pivot_table(index=['chr', 'start', 'end'], columns='method', values='regions', fill_value=0)
# # Reset index to convert to a flat DataFrame
# pivot_table.reset_index(inplace=True)

# # List of columns to exclude
# exclude_columns = ['chr', 'start', 'end']
# # Convert non-NA values to True and NA values to False, excluding specified columns
# boolean_region_overlap = pivot_table.copy()
# # replace 0s with NA
# boolean_region_overlap.replace(0, np.nan, inplace=True)
# # replace with boolean values
# boolean_region_overlap[boolean_region_overlap.columns.difference(exclude_columns)] = boolean_region_overlap[boolean_region_overlap.columns.difference(exclude_columns)].applymap(lambda x: pd.notna(x))

# boolean_region_overlap.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_region_overlap_no_argweaver.bed', sep='\t', index=False)
# # drop chrom, start, end
# boolean_region_overlap_no_bed = boolean_region_overlap.drop(['chr', 'start', 'end'], axis=1)
# boolean_region_overlap_no_bed.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_region_overlap_no_argweaver_no_bed', sep='\t', index=False)
# boolean_region_overlap

,ibdmix,archaicseeker,archie,sprime,vernot_2016,sankararaman_2014,sankararaman_2016_1,sankararaman_2016_2,skov_2020,steinruecken_2018,sarge,length
0,False,True,False,False,False,False,False,False,False,False,False,4580
1,False,True,False,False,False,False,False,False,False,False,False,5610
2,False,True,False,False,False,False,False,False,False,False,False,4454
3,False,True,False,False,False,False,False,False,False,False,False,22746
4,False,True,False,False,False,False,False,False,False,False,False,9495
...,...,...,...,...,...,...,...,...,...,...,...,...
364386,False,False,False,False,False,False,True,False,False,False,False,109687
364387,False,False,False,False,False,False,True,False,False,False,False,75368
364388,False,False,False,False,False,False,True,False,False,False,False,37507
364389,False,False,False,False,False,False,True,False,False,False,False,3066


#### back to including argweaver

In [10]:
chen_2020_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged.bg', sep='\t')
hubisz_2020_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs/union_merged.bg', sep='\t')
yuan_2021_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/union_merged.bg', sep='\t')
durvasula_2019_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/union_merged.bg', sep='\t', names=['chrom', 'start', 'end', 'CEU'])
browning_2018_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/union_merged.bg', sep='\t')
vernot_2016_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/union_merged.bg', sep='\t')
sankararaman_2014_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/union_merged.bg', sep='\t')
sankararaman_2016_1_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/union_merged.bg', sep='\t')
sankararaman_2016_2_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/union_merged.bg', sep='\t')
steinruecken_2018_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/union_merged.bg', sep='\t')
schaefer_2021_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/union_merged.bg', sep='\t')
skov_2020_pops = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/union_merged_highest_score.bg', sep='\t')
skov_2020_pops.rename(columns={'highest_score': 'Icelandic'}, inplace=True)

# Combine each separate file into one massive file
all_methods_concat_pops = pd.concat([chen_2020_pops, hubisz_2020_pops, yuan_2021_pops, durvasula_2019_pops, browning_2018_pops, vernot_2016_pops, sankararaman_2014_pops, sankararaman_2016_1_pops, sankararaman_2016_2_pops, skov_2020_pops, steinruecken_2018_pops, schaefer_2021_pops], axis=0)
all_methods_concat_pops = all_methods_concat_pops.fillna(0)
all_methods_concat_pops.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops.bed', sep='\t', index=False)

# add '#' for header
!sed -i '1,1s/^/#/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops.bed 
# collapse extra tabs
!tr -s '\t' '\t' < {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_header_tabs_collapse.bed
 # remove extra tabs and sort file
!sed 's/\t$//' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_header_tabs_collapse.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_tab_removed.bed
# remove header
!sed -e '1d' -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_tab_removed.bed
!sort -k 1,1 -k2,2n {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_tab_removed.bed > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops.bed
# re-add header after sorting
# Extract the first line from the source file
!read -r first_line<$HOMEDIR/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_tab_removed.bed
# add header line
!cp {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops.bed {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_header.bed
!sed -i '1s/^/#chrom\tstart\tend\tEAS\tEUR\tAMR\tSAS\tAFR\tAfrica\tOceania\tWestEurasia\tCEU\tASN\tPNG\tCentralAsia\tIcelandic\tCHBS\tEastAsia\tAfrica2\tAmerica\tCentralAsiaSiberia\tSouthAsia\n/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_header.bed
# had to hard code because sed was giving me such a hard time inserting first line from another file

# remove intermediate files
!rm {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_header_tabs_collapse.bed \
{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods_concat_pops_tab_removed.bed

In [2]:
## region level - made sorted_union_merged_chr_prefix.bg
methods_dirs = ['chen_2020', 'yuan_2021', 'durvasula_2019', 'steinruecken_2018', 'skov_2020', 'schaefer_2021', 'hubisz_2020', 'browning_2018', 'vernot_2016', 'sankararaman_2014', 'sankararaman_2016_1', 'sankararaman_2016_2']
# Use bedtools complement to get the non-introgressed regions for each method
for method in methods_dirs:
    if 'sankararaman' in method:
        method_name = method.replace('_', '', 1)
    else:
        method_name = method.replace('_', '')
    # specify method dir
    print(method)
    method_dir = f'/introgression_methods/cleaned/introgression_tools/{method}'
    # only keep chrom, start, stop
    !cut -f1,2,3 {HOMEDIR}/{method_dir}/bedgraphs/union_merged.bg > {HOMEDIR}/{method_dir}/bedgraphs/union_merged_prefix.bg
    # remove header
    !sed -e '1d' -i {HOMEDIR}/{method_dir}/bedgraphs/union_merged_prefix.bg
    # add chr prefix
    !sed -e 's/^/chr/' {HOMEDIR}/{method_dir}/bedgraphs/union_merged_prefix.bg > {HOMEDIR}/{method_dir}/bedgraphs/union_merged_chr_prefix.bg
    # sort both files
    !sort -k1,1 /wynton/group/capra/data/hg19_fasta/2022-03-14/hg19.fa.fai > {HOMEDIR}/introgression_methods/data/sorted_hg19.fa.fai
    !sort -k1,1 -k2,2n {HOMEDIR}/{method_dir}/bedgraphs/union_merged_chr_prefix.bg > {HOMEDIR}/{method_dir}/bedgraphs/sorted_union_merged_chr_prefix.bg
    # run bedtools complement
    # os.system(f'{BEDTOOLSDIR}/bedtools complement -L -i {HOMEDIR}/{method_dir}/bedgraphs/sorted_union_merged_chr_prefix.bg -g {HOMEDIR}/introgression_methods/data/sorted_hg19.fa.fai > {HOMEDIR}/{method_dir}/bedgraphs/complement.bg')
    # # subtract gaps
    # os.system(f'{BEDTOOLSDIR}/bedtools subtract -a {HOMEDIR}/{method_dir}/bedgraphs/complement.bg -b {HOMEDIR}/introgression_methods/data/hg19_gaps.txt > {HOMEDIR}/{method_dir}/complement_gaps_removed.bg')
    # # keep desert regions only in filtered regions from the Nean genome - min filtering
    # os.system(f'{BEDTOOLSDIR}/bedtools intersect -a {HOMEDIR}/{method_dir}/complement_gaps_removed.bg -b {HOMEDIR}/introgression_methods/data/altai_minimal_filters/AltaiNea.map35_50.MQ30.Cov.indels.TRF_chr_prefix.bed > {HOMEDIR}/{method_dir}/complement_gaps_removed_altai_filtered_min.bg')
    # # keep desert regions only in filtered regions from the Nean genome - more extensive filtering from Altai
    # os.system(f'{BEDTOOLSDIR}/bedtools intersect -a {HOMEDIR}/{method_dir}/complement_gaps_removed.bg -b {HOMEDIR}/introgression_methods/data/vindija_filtered_regions/altai_filtered_chr_prefix.bed > {HOMEDIR}/{method_dir}/complement_gaps_removed_altai_filtered.bg')

chen_2020
yuan_2021
durvasula_2019
steinruecken_2018
skov_2020
schaefer_2021
hubisz_2020
browning_2018
vernot_2016
sankararaman_2014
sankararaman_2016_1
sankararaman_2016_2


In [12]:
# make a version with different universe: each unique combination of detected regions is a separate region
## THIS ONE IS THE FINALIZED ONE
!{BEDTOOLSDIR}/bedtools intersect -a {HOMEDIR}/introgression_methods/cleaned/introgression_tools/overlap_bed_only_no_header_chr_prefix.bg \
    -b {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/sorted_union_merged_chr_prefix.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/sorted_union_merged_chr_prefix.bg \
    -C -names ibdmix argweaverd archaicseeker archie sprime vernot_2016 sankararaman_2014 sankararaman_2016_1 sankararaman_2016_2 skov_2020 steinruecken_2018 sarge \
   > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/region_intersect_for_boolean.bed

## Generate file with introgression across methods (no ARGWeaver-D), subsetted to a shared length across them, keeping the highest score


In [6]:
# remove header from 
methods_dirs = ['chen_2020', 'yuan_2021', 'durvasula_2019', 'steinruecken_2018', 'skov_2020', 'schaefer_2021', 'hubisz_2020', 'browning_2018', 'vernot_2016', 'sankararaman_2014', 'sankararaman_2016_1', 'sankararaman_2016_2']
# Use bedtools complement to get the non-introgressed regions for each method
for method in methods_dirs:
    method_dir = f'/introgression_methods/cleaned/introgression_tools/{method}'
    with open(f'{HOMEDIR}/{method_dir}/bedgraphs/union_merged_highest_score.bg', 'r') as f:
        first_line = f.readline()
        if 'chrom' in first_line.lower():
            !sed -e '1d' -i {HOMEDIR}/{method_dir}/bedgraphs/union_merged_highest_score.bg

In [14]:
# empty dict to hold proportions
proportions = {}
for method in methods_dirs:
    df = pd.read_csv(f'{HOMEDIR}/{method_dir}/bedgraphs/union_merged_highest_score.bg', sep='\t', low_memory=False, names=['chrom', 'Start', 'End', 'Score'])
    # compute length
    df['length'] = df['End'] - df['Start']
    # compute the minumum amount of introgression across people
    min_sum = df['length'].sum().min()
    # add the minimum amount to the dict
    proportions[method] = min_sum
# get the smallest value to use as length_threshold
length_threshold = min(proportions.values())
print(proportions, length_threshold)

{'chen_2020': 1141804104, 'yuan_2021': 1141804104, 'durvasula_2019': 1141804104, 'steinruecken_2018': 1141804104, 'skov_2020': 1141804104, 'schaefer_2021': 1141804104, 'hubisz_2020': 1141804104, 'browning_2018': 1141804104, 'vernot_2016': 1141804104, 'sankararaman_2014': 1141804104, 'sankararaman_2016_1': 1141804104, 'sankararaman_2016_2': 1141804104} 1141804104


In [22]:
boolean_overlap_shared_length_highest_scores = tools.generate_overlap(input_files_string=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/union_merged_highest_score.bg',
                                           input_files_names_str='ibdmix archaicseeker archie sprime vernot_2016 sankararaman_2014 sankararaman_2016_1 sankararaman_2016_2 skov_2020 steinruecken_2018 sarge',
                                           output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_shared_length_highest_scores.bed',
                                           boolean=True, autosomes=True, length_threshold=719560342)

In [12]:
# also a version with EUR regions only
boolean_overlap_shared_length_highest_scores_EUR = tools.generate_overlap(input_files_string=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged_highest_score.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/merged_EUR.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/merged_EUR.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/merged_EUR.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/merged_EUR.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/merged_EUR.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/merged_EUR.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/merged_EUR.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/merged_Icelandic.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/merged_CEU.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/merged_WestEurasia.bg',
                                           input_files_names_str='ibdmix archaicseeker archie sprime vernot_2016 sankararaman_2014 sankararaman_2016_1 sankararaman_2016_2 skov_2020 steinruecken_2018 sarge',
                                           output_file_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/boolean_overlap_shared_length_highest_scores_EUR.bed',
                                           boolean=True, autosomes=True, length_threshold=719560342)

## Generate previously computed deserts

Available deserts:
- Chen 2020 (in Table S8)
- Skov 2020 (Supplementary Dataset 4, 41586_2020_2225_MOESM6_ESM, "This dataset reports the chrom, desert number, start, mean windows called as archaic, number of called and total number of genes.")
- Vernot 2016 (In Chen 2020 Table S8)
- Sankararaman 2016 (previously emailed Dr. Sankararaman)
- Yuan 2021 (Supplementary Data 6)

In [3]:
## manually write .bed files for Chen 2020 and Vernot deserts:
chen_2020_deserts = {
    'chrom': [1, 3, 7, 8],
    'start': [105400000, 74100000, 106200000, 49400000],
    'end': [120600000, 89300000, 123200000, 66500000]
}

chen_2020_deserts_df = pd.DataFrame(chen_2020_deserts)
chen_2020_deserts_df.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/original_deserts.bg', header=False, sep='\t', index=False)

# save version with chr prefix
chen_2020_deserts_df['chrom'] = 'chr' + chen_2020_deserts_df['chrom'].astype(str)
chen_2020_deserts_df.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/original_deserts_chr_prefix.bg', header=False, sep='\t', index=False)

vernot_2016_deserts = {
    'chrom': [1, 2, 3, 7, 8, 18],
    'start': [102200000, 201100000, 76500000, 106300000, 53900000, 25000000],
    'end': [114900000, 211500000, 90500000, 124700000, 66000000, 41800000]
}

vernot_2016_deserts_df = pd.DataFrame(vernot_2016_deserts)
vernot_2016_deserts_df.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/original_deserts.bg', header=False, sep='\t', index=False)

# save version with chr prefix
vernot_2016_deserts_df['chrom'] = 'chr' + vernot_2016_deserts_df['chrom'].astype(str)
vernot_2016_deserts_df.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/original_deserts_chr_prefix.bg', header=False, sep='\t', index=False)

# load in ArchaicSeeker2 deserts file (supplementary figure 6)
yuan_2021_deserts=pd.read_excel(f'{HOMEDIR}/introgression_methods/data/archaicseeker2.0_yuan_et_al_2021/41467_2021_26503_MOESM9_ESM.xlsx')
# save only as bed file with chrom, start, end
yuan_2021_deserts[['Chromosome', 'Start (bp)', 'End (bp)']].to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/original_deserts.bg', header=False, sep='\t', index=False)

In [33]:
# Sankararaman deserts are already in a cleaned .bed format, copy over to cleaned directory
! cp {HOMEDIR}/introgression_methods/data/sankararaman_2016/deserts/sankararaman16_nean-deserts_hg19.bed {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/original_deserts_chr_prefix.bg

sankararaman_2016_deserts = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/original_deserts_chr_prefix.bg', sep='\t', names=['chrom', 'start', 'end'])
sankararaman_2016_deserts['chrom'] = sankararaman_2016_deserts['chrom'].str.replace('chr', '')
sankararaman_2016_deserts.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/original_deserts.bg', header=False, sep='\t', index=False)

In [79]:
## Load in Skov 2020 deserts file -- Need to liftover from hg38 to hg19
skov_2020_deserts = pd.read_csv(f'{HOMEDIR}/introgression_methods/data/skov_2020/41586_2020_2225_MOESM6_ESM.txt', sep='\t')
skov_2020_deserts = skov_2020_deserts[['chrom', 'start', 'end']]
skov_2020_deserts['start'] = np.int64(skov_2020_deserts['start'])
skov_2020_deserts['end'] = np.int64(skov_2020_deserts['end'])
# add chr prefix
skov_2020_deserts['chrom'] = 'chr' + skov_2020_deserts['chrom']
#skov_2020_deserts.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/original_deserts_hg38.bg', header=False, sep='\t', index=False)

In [85]:
skov_2020_deserts = pd.read_csv(f'{HOMEDIR}/introgression_methods/data/skov_2020/41586_2020_2225_MOESM6_ESM.txt', sep='\t')

np.sum(skov_2020_deserts.mean_called)


0

In [20]:
# run liftover
!/wynton/group/capra/bin/liftOver/liftOver \
{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/original_deserts_hg38.bg \
{HOMEDIR}/introgression_methods/data/skov_2020/hg38_to_hg19_liftover/hg38ToHg19.over.chain.gz \
{HOMEDIR}/introgression_methods/data/skov_2020/hg38_to_hg19_liftover/original_deserts_hg19.bed \
{HOMEDIR}/introgression_methods/data/skov_2020/hg38_to_hg19_liftover/deserts_unlifted.bed

# copy final lifted file to cleaned directory
!cp {HOMEDIR}/introgression_methods/data/skov_2020/hg38_to_hg19_liftover/original_deserts_hg19.bed {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/original_deserts_chr_prefix.bg
# remove 'chr' prefix in cleaned file
skov_2020_deserts = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/original_deserts_chr_prefix.bg', sep='\t', names=['chrom', 'start', 'end'])
skov_2020_deserts['chrom'] = skov_2020_deserts['chrom'].str.replace('chr', '')
skov_2020_deserts.to_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/original_deserts.bg', header=False, sep='\t', index=False)

Reading liftover chains
Mapping coordinates


After LiftOver, we went from 281 desert regions in hg38 -> 268 desert regions in hg19.

### Combined desert file

In [9]:
# sort the bg file
bedops_dir='/wynton/home/cbi/shared/software/CBI/bedops-2.4.41/bin/'
!{bedops_dir}/sort-bed --unique {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/original_deserts.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/sorted_original_deserts.bg
!{bedops_dir}/sort-bed --unique {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/original_deserts.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/sorted_original_deserts.bg
!{bedops_dir}/sort-bed --unique {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/original_deserts.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/sorted_original_deserts.bg
!{bedops_dir}/sort-bed --unique {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/original_deserts.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/sorted_original_deserts.bg
!{bedops_dir}/sort-bed --unique {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/original_deserts.bg > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/sorted_original_deserts.bg

# add 1s
!sed -i 's/$/&\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/sorted_original_deserts.bg
!sed -i 's/$/&\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/sorted_original_deserts.bg
!sed -i 's/$/&\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/sorted_original_deserts.bg
!sed -i 's/$/&\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/sorted_original_deserts.bg
!sed -i 's/$/&\t1/' {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/sorted_original_deserts.bg

# generate a combined file
!$BEDTOOLSDIR/bedtools unionbedg -i {HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/sorted_original_deserts.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/sorted_original_deserts.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/sorted_original_deserts.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/sorted_original_deserts.bg \
    {HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/sorted_original_deserts.bg \
    -header -names sankararaman_2016_1 skov_2020 chen_2020 vernot_2016 yuan_2021 > {HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_deserts.bed

# Generate files for rGREAT

Here I generate the overall Neanderhal introgression background file, where any region considered introgressed, regardless of method, is included.

In [ ]:
# background file containing all regions that were considered introgressed (all_methods.bed file that we previously created)
# need to remove the last columns
# directory to store data
!mkdir {HOMEDIR}/introgression_methods/data/GREAT_analysis
!mkdir {HOMEDIR}/introgression_methods/data/GREAT_analysis/input
overlap = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods.bed', sep='\t', low_memory=False)
nean_background = overlap[['chrom', 'start', 'end']]
nean_background['chrom'] = 'chr' + nean_background['chrom'].astype(str)
nean_background.to_csv(f'{HOMEDIR}/introgression_methods/data/GREAT_analysis/nean_background.bed', sep='\t', index=False, header=False)
# merge overlapping entries that were previously separated due to counting the overlap between different sets of methods, now we are not worried about a score associated with each introgressed region
!$BEDTOOLSDIR/bedtools merge -i {HOMEDIR}/introgression_methods/data/GREAT_analysis/nean_background.bed > {HOMEDIR}/introgression_methods/data/GREAT_analysis/nean_background_merged.bed
# remove unmerged file
!rm {HOMEDIR}/introgression_methods/data/GREAT_analysis/nean_background.bed

I will generate test files for different introgressed regions I want to test, and background files containing all regions that contain introgression.

Here is a function to generate non-overlapping bed files with each method as the test set, and all detected Neanderthal introgressed regions as the background set (the background set must contain identical test set entries, plus other regions that are part of the background).

In [9]:
# GREAT requires background file to contain exactly the same entries as the test, plus extra background regions as separate entries
# subtract all nean introgression regions - test introgression set to get extra introgressed regions
def GREAT_background_files(method_name=None, test_df=None, test_df_path=None):
    if test_df_path is not None:
        df = pd.read_csv(f'{test_df_path}', sep='\t', low_memory=False, names=['chrom', 'start', 'end'])
    elif test_df is not None:
        df = test_df
    # add chr prefix if the column contains integers
    if df['chrom'].dtype != 'str':
        df['chrom'] = 'chr' + df['chrom'].astype(str)
    test_bed = df[['chrom', 'start', 'end']]
    test_bed.to_csv(f'{HOMEDIR}/introgression_methods/data/GREAT_analysis/{method_name}_GREAT.bed', sep='\t', index=False, header=False)
    !$BEDTOOLSDIR/bedtools merge -i {HOMEDIR}/introgression_methods/data/GREAT_analysis/{method_name}_GREAT.bed > {HOMEDIR}/introgression_methods/data/GREAT_analysis/input/{method_name}_GREAT_merged.bed
    # remove unmerged file
    # subtract test regions from full nean background file 
    !$BEDTOOLSDIR/bedtools subtract -a {HOMEDIR}/introgression_methods/data/GREAT_analysis/nean_background_merged.bed \
    -b {HOMEDIR}/introgression_methods/data/GREAT_analysis/input/{method_name}_GREAT_merged.bed > {HOMEDIR}/introgression_methods/data/GREAT_analysis/background_sub_{method_name}.bed
    # append original test regions to nean introgressed regions (subtracted test from full nean background file)
    !cat {HOMEDIR}/introgression_methods/data/GREAT_analysis/input/{method_name}_GREAT_merged.bed {HOMEDIR}/introgression_methods/data/GREAT_analysis/background_sub_{method_name}.bed > {HOMEDIR}/introgression_methods/data/GREAT_analysis/{method_name}_background.bed
    # re-sort regions
    !$BEDTOOLSDIR/bedtools sort -i {HOMEDIR}/introgression_methods/data/GREAT_analysis/{method_name}_background.bed > {HOMEDIR}/introgression_methods/data/GREAT_analysis/input/{method_name}_background_GREAT.bed

In [11]:
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/chen_2020/bedgraphs/union_merged_prefix.bg', method_name='ibdmix')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/hubisz_2020/bedgraphs/union_merged_prefix.bg', method_name='argweaverd')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/yuan_2021/bedgraphs/union_merged_prefix.bg', method_name='archaicseeker')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/durvasula_2019/bedgraphs/union_merged_prefix.bg', method_name='archie')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/browning_2018/bedgraphs/union_merged_prefix.bg', method_name='sprime')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/vernot_2016/bedgraphs/union_merged_prefix.bg', method_name='vernot2016')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2014/bedgraphs/union_merged_prefix.bg', method_name='sankararaman2014')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_1/bedgraphs/union_merged_prefix.bg', method_name='sankararaman2016_1')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/sankararaman_2016_2/bedgraphs/union_merged_prefix.bg', method_name='sankararaman2016_2')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/skov_2020/bedgraphs/union_merged_prefix.bg', method_name='skov2020')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/steinruecken_2018/bedgraphs/union_merged_prefix.bg', method_name='steinruecken2018')
GREAT_background_files(test_df_path=f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/schaefer_2021/bedgraphs/union_merged_prefix.bg', method_name='sarge')

In [29]:
# generate files for GREAT input
GREAT_background_files(sarge, 'sarge') # no enrichments under default parameters
GREAT_background_files(ibdmix, 'ibdmix') # no enrichments under default parameters
GREAT_background_files(archaicseeker, 'archaicseeker') # no enrichments under default parameters
GREAT_background_files(sankararaman_2016_1, 'sankararaman2016_1') # no enrichments under default parameters
GREAT_background_files(sankararaman_2016_2, 'sankararaman2016_2') # no enrichments under default parameters
GREAT_background_files(sprime, 'sprime') # no enrichments under default parameters
GREAT_background_files(vernot_2016, 'vernot2016') # no enrichments under default parameters
GREAT_background_files(argweaverd, 'argweaverd') # HAS ENRICHMENTS: GPHN is top gene, scaffolding molecule at inhibitory neuron synapses
GREAT_background_files(sankararaman_2014, 'sankararaman2014') # no enrichements under default parameters
GREAT_background_files(skov_2020, 'skov2020') # HAS ENRICHMENTS: axonal growth cone, amino acid binding, pancreas size (in mouse)
GREAT_background_files(steinruecken_2018, 'steinruecken2018') # no enrichments under default parameters
GREAT_background_files(archie, 'archie') # HAS ENRICHMENTS: hypoxia, respiratory birse, cell migration in hindbrain :) and lots of other stuff

I want to test if regions overlapping across all 12 methods have specific genomic elements compared to the background. Additionally, I will repeat the test but excluding ArchIE and ARGweaver-D which have small test sets to look at regions overlapping over the remaining 10 methods (high-confidence Neanderthal introgressed methods).

In [24]:
# Generate bed file with all 12 methods overlapping
overlap_12_methods = overlap.dropna()
overlap_12_methods = overlap_12_methods[['chrom', 'start', 'end']]

GREAT_background_files('overlap_12_methods', test_df=overlap_12_methods) # HAS ENRICHMENTS: lots of telomere processes, ATPase activity

In [26]:
overlap_10_methods = overlap.drop(columns=['archie', 'argweaverd'])
overlap_10_methods = overlap_10_methods.dropna()
overlap_10_methods_great = overlap_10_methods[['chrom', 'start', 'end']]

GREAT_background_files('overlap_10_no_archie_argweaver', test_df=overlap_10_methods_great) # no enrichments under default parameters

I will also generate GREAT files where at least 10 methods support a region (including ArchIE and ARGweaver-D)

In [ ]:
overlap = pd.read_csv(f'{HOMEDIR}/introgression_methods/cleaned/introgression_tools/all_methods.bed', sep='\t', low_memory=False)
overlap.replace(0, np.nan, inplace=True)
# List of columns to exclude to make boolean
exclude_columns = ['chrom', 'start', 'end', 'length']
overlap[overlap.columns.difference(exclude_columns)] = overlap[overlap.columns.difference(exclude_columns)].apply(lambda x: pd.notna(x))

# methods cols to check for boolean
columns_to_check = [item for item in overlap.columns if item not in exclude_columns]

# keep rows with at least 10 TRUE values
mask = overlap[columns_to_check].fillna(False).astype(bool).sum(axis=1) >= 10

# filter the original DataFrame based on the mask
overlap_10_more = overlap[mask]
overlap_10_more # number matches the num support plot

466317072

In [10]:
GREAT_background_files('overlap_10_more_methods', test_df=overlap_10_more)